# Search: Path finding

To goal is to find a path through a maze from a given start state $S$ to a given goal state $G$. Here is a small
example maze:

```
XXXXXXX
XSXGX X
X X   X
X XX  X
X     X
XXXXXXX
```

Implement the following search algorithms for solving different mazes:

* Depth-first search
* Breadth-first search
* Greedy best-first search
* A* search (using Manhattan distance from the current position to the goal as the heuristic function)

Run each of the above algorithms on the small maze, medium maze, big maze, and the open maze. For each problem instance and each search algorithm, report the following:

* The solution and its path cost
* Number of nodes expanded
* Maximum tree depth searched
* Maximum size of the frontier.

You can display the solution by putting a `.` in every maze square visited on the path and a `*` for the squares on the final path.